#Operations Research, IE33150
#**Case: “Red Tomato Tools”**


##Parametrs Definition
* $T$: The set of months planning horizon , \{"January", "Febreuary", "March","April","May", "June"\}
* $D_t \equiv $  Demand forecast in each period  $t$, $t\in T$.
* $MC \equiv $ Materials cost per unit  \$ / unit.
* $IHC \equiv $ The inventory holding cost in  \$ / unit/month.
* $MCS \equiv $ The marginal cost of stock out \$ / unit/month. **In cost accounting, stockout costs represent what you lose when an item is out of stock.**
* $HTC \equiv $ The hiring and training costs in  \$ / worker.
* $LOC \equiv $ The layoff cost in  \$ / worker.**Layoffs typically mean companies have to pay severance costs, which vary widely by occupation and industry.**
* $LH \equiv $ The labour hours required to produce one unit in   Hrs/ unit.
* $RC \equiv $ The regular time cost in  \$ / hour.
* $OC \equiv $ The over time cost in  \$ / hour.
* $SC \equiv $ The sub contracting cost in  \$ / unit.



##Decison Variables
\begin{align*}
    W_{t} & \equiv \text {Workforce size for month t} \\
    H_{t} & \equiv \text {Number of employees hired at the begining of month t} \\
    L_{t} & \equiv \text {Number of employees laid off at the begining of month t} \\
    P_{t} & \equiv \text {Number of units produced in month t, Production in month t} \\
    I_{t} & \equiv \text {Inventory at the end of month t} \\
    S_{t} & \equiv \text {Number of units stocked out at the end of month t } \\
    C_{t} & \equiv \text {Number of units subcontarcted for month t} \\
    O_{t} & \equiv \text {Number of overtime hours  worked in month t} \\  
\end{align*}

##Linear Programme
We have been asked to minimize the total cost, total cost includes:
* Workforce payment costs.(Working days in each month* Working hours in each day * RC)
* Hiring & training costs.
* Downsizing costs (Lay off costs).
* Overtime costs.
* Inventory costs.
* Stock-out costs.
* Production costs
* Subcontracting costs.
\begin{align*}
    \min \hspace{10pt} & \sum_{t \in T}(20*8*RC) W_t + \sum_{t \in T}(HTC) H_t + \sum_{t \in T}(LOC)L_t +\sum_{t \in T}(OC) O_t +\sum_{t \in T}(IHC) I_t  +\sum_{t \in T}(MCS) S_t + \sum_{t \in T}(MC) P_t + \sum_{t \in T}(SC) C_t  \\
    \text{s.t.} \hspace{7pt}
    & W_t= W_{t-1} + H_t - L_t & \forall t \in T\\
    & P_t \le \frac {(20*8)}{LH} W_{t} +\frac {O_t}{LH } & \forall t \in T\\
    & I_{t-1} + P_t +C_t= D_t + S_{t-1} + I_t -S_t & \forall t \in T\\
    & O_t \le (10)W_t & \forall t \in T\\
    & W_0= 80\\
    & I_0= 1000\\
    & S_0= 0\\
    & S_6= 0\\
    & I_6\ge 500\\
    & W_{t}, H_{t}, L_{t}, P_{t}, I_{t}, S_{t}, C_{t}, O_{t}\ge 0 & \forall t \in T\\
  \end{align*}



# Constarints Explanation
**First Constraint:** Balance of workers constarints, number of workers employed at the end of a particular month is equal to the number of workers employed at the begining of the successive month.
Only hiring and layoff processes are responsible for changing the number of workers at a certain month.\
**Second Constraint:** Not to exceeed the predifined production limit,production for each month can not exceed the capacity , remember each one unit neeeds 4 hours to be produced, thus the total number of units that can be produced is equal to the available working hours devided by 4.
Available working hours, (20*8* Number of workers) for the regular time and ($O_t$) for the overtime.\
**Third Constraint:** Inventory balance constraints, All units that we have in hand at the begining of month t plus all what we produce/receive in  that specific month should be equal to what customers get (either as that month demand or previous month stockout) plus what remains as invetory and what we decide not to deliver. \
**Fourth Constraint:** Overtime limit, no employee works more than 10 hours of overtime per month.\
**Fifth,Sixth & Seventh Constraints:**Initial Conditions.\
**Eighth & ninth Constraints:** Final Conditions.\
**Tenth Constraint:** Non-Negativity Constraints.






In [ ]:
!pip install cplex
!pip install docplex

     |████████████████████████████████| 30.9MB 147kB/s 
     |████████████████████████████████| 614kB 4.1MB/s 
  Created wheel for docplex: filename=docplex-2.20.204-cp37-none-any.whl size=675362 sha256=994ea94ada9e8e412c4203d6b94a7596ec55bd094a564c5a6e27cd7ff0df17f6
  Stored in directory: /root/.cache/pip/wheels/ae/2c/e2/a099ebb6fda8adeba9c5fc2e25659d195ad2f5c6cc5fb75fd4
Successfully built docplex


In [ ]:
# First impot the Model class from docplex.mp
from docplex.mp.model import Model

In [ ]:
# Create one model instance, with a name
m = Model (name="Red Tomato Tools_Case Study")

In [ ]:
# Defining the parametrs of the problem statement
Months = [1,2,3,4,5,6]
Demand_Forecast= {1: 1600,
                  2: 3000,
                  3: 3200,
                  4: 3800,
                  5: 2200,
                  6: 220} #This value has been updated

# Parameters Table 2, Costs for Red Tomato.
MC  = 10
IHC = 8  #This value has been updated
MCS = 15  #This value has been updated
HTC = 300
LOC = 500
LH  = 4
RC  = 4
OC  = 6
SC  = 30



In [ ]:
# Defining the decison variabels of the problem
W_t = m.integer_var_dict(range (0,7),lb=0,ub=None,name=" Workforce Size Per Month %s")
H_t = m.integer_var_dict(Months,lb=0,ub=None,name=" Number of employees hired at the begining of Month %s")
L_t = m.integer_var_dict(Months,lb=0,ub=None,name=" Number of employees laid off at the begining of Month %s")
P_t = m.integer_var_dict(Months,lb=0,ub=None,name=" Number of units produced during  Month %s")
I_t = m.integer_var_dict(range (0,7),lb=0,ub=None,name=" Inventory at the end of Month %s")
S_t = m.integer_var_dict(range (0,7),lb=0,ub=None,name=" Number of units stocked out at the end of Month %s")
C_t = m.integer_var_dict(Months,lb=0,ub=None,name=" Number of units subcontracted for Month %s")
O_t = m.continuous_var_dict(Months,lb=0,ub=None,name=" Number of overtime hours worked in month %s")

In [ ]:
# Defining the objective function of the problem
# Option (1): minimizing the total cost incurred, this way the objective value will be equal to 422660.000

"""m.minimize (m.sum((20*8*RC) *W_t[i]for i in Months) + m.sum((HTC) *H_t[i]for i in Months) +\
            m.sum((LOC) *L_t[i]for i in Months)+ m.sum((OC) *O_t[i]for i in Months) +\
            m.sum((IHC) *I_t[i]for i in Months) + m.sum((MCS) *S_t[i]for i in Months)+\
            m.sum((MC)  *P_t[i]for i in Months) +m.sum((SC) *C_t[i]for i in Months))"""



'm.minimize (m.sum((20*8*RC) *W_t[i]for i in Months) + m.sum((HTC) *H_t[i]for i in Months) +            m.sum((LOC) *L_t[i]for i in Months)+ m.sum((OC) *O_t[i]for i in Months) +            m.sum((IHC) *I_t[i]for i in Months) + m.sum((MCS) *S_t[i]for i in Months)+            m.sum((MC)  *P_t[i]for i in Months) +m.sum((SC) *C_t[i]for i in Months))'

In [ ]:
# Another technique can be used, by which an expressionn will be created to calculte each cost seperatley
Regular_Time_Labour_Cost= m.sum((20*8*RC) *W_t[i]for i in Months)
Hiring_Training_Cost= m.sum((HTC) *H_t[i]for i in Months)
Downsizing_Cost=  m.sum((LOC) *L_t[i]for i in Months)
Over_Time_Labour_Cost= m.sum((OC) *O_t[i]for i in Months)
Inventory_Holding_Cost= m.sum((IHC) *I_t[i]for i in Months)
Material_Cost= m.sum((MCS) *S_t[i]for i in Months)
Production_Cost= m.sum((MC)  *P_t[i]for i in Months)
Subcontracting_Cost= m.sum((SC) *C_t[i]for i in Months)

In [ ]:

m.minimize(Regular_Time_Labour_Cost+Hiring_Training_Cost+Downsizing_Cost+Over_Time_Labour_Cost+\
           Inventory_Holding_Cost+Material_Cost+Production_Cost+Subcontracting_Cost)

In [ ]:

# Option (2): maximizing the net profit (Revenue - Cost), this way the objective value will be equal to 217340.000
# The two optiions will give the exact same values for the decsion variabels at optimality.
"""m.maximize(m.sum(Demand_Forecast.values() )*40 -(m.sum((20*8*RC) *W_t[i]for i in Months) + m.sum((HTC) *H_t[i]for i in Months) +\
            m.sum((LOC) *L_t[i]for i in Months)+ m.sum((OC) *O_t[i]for i in Months) +\
            m.sum((IHC) *I_t[i]for i in Months) + m.sum((MCS) *S_t[i]for i in Months)+\
            m.sum((MC)  *P_t[i]for i in Months) +m.sum((SC) *C_t[i]for i in Months)))"""


'm.maximize(m.sum(Demand_Forecast.values() )*40 -(m.sum((20*8*RC) *W_t[i]for i in Months) + m.sum((HTC) *H_t[i]for i in Months) +            m.sum((LOC) *L_t[i]for i in Months)+ m.sum((OC) *O_t[i]for i in Months) +            m.sum((IHC) *I_t[i]for i in Months) + m.sum((MCS) *S_t[i]for i in Months)+            m.sum((MC)  *P_t[i]for i in Months) +m.sum((SC) *C_t[i]for i in Months)))'

In [ ]:
# Defining the constraints of the problem

# Balance of Workers
m.add_constraints_(W_t [i] == W_t[i-1]+ H_t [i] - L_t [i]for i in Months)

# Production Limit Constraints
m.add_constraints_(P_t [i] <= ((20*8)*W_t[i]/(LH))+ (O_t[i]/LH) for i in Months)

# Balance of Inventory
m.add_constraints_(I_t [i-1] + P_t [i] + C_t [i]  == Demand_Forecast[i]+ S_t [i-1] + I_t [i] - S_t[i]for i in Months)

# Overtime Limit
m.add_constraints_(O_t [i] <= (10)* W_t[i] for i in Months)

# Initial Conditions Constraints
m.add_constraint_(W_t[0]== 80)
m.add_constraint_(I_t[0]== 1000)
m.add_constraint_(S_t[0]== 0)

# Final Conditions Constarints
m.add_constraint_(S_t[6]== 0)
m.add_constraint_(I_t[6]>= 500)



In [ ]:
# Print the model information
m.print_information()

Model: Red Tomato Tools_Case Study
 - number of variables: 51
   - binary=0, integer=45, continuous=6
 - number of constraints: 29
   - linear=29
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [ ]:
m.solve (log_output =True)

print (m.solve_details.status)

m.print_solution()

Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 801760.000000 after 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 6 rows and 5 columns.
MIP Presolve added 12 rows and 12 columns.
Reduced MIP has 35 rows, 58 columns, and 119 nonzeros.
Reduced MIP has 0 binaries, 52 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.05 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 12 rows and 12 columns.
MIP Presolve added 12 rows and 12 columns.
Reduced MIP has 35 rows, 58 columns, and 119 nonzeros.
Reduced MIP has 0 binaries, 52 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.07 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 2 threads.
Root relaxation solution time = 0.00 sec. (0.10 ticks)

        Nodes                                         Cut